# Data Consolidation & Preprocessing

Combine the two training datasets (`Training_part1.csv` and `Training_part2.csv`) using the `id` column, preprocess the data, split into train/test sets, and save the result.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Define file paths
file_path1 = 'data/Training_part1.csv'
file_path2 = 'data/Training_part2.csv'

# Load the datasets
df1 = pd.read_csv(file_path1, sep=";")
df2 = pd.read_csv(file_path2, sep=";")

# Display the first few rows and info of each dataframe to understand their structure
print("First DataFrame (df1) Head:")
print(df1.head())
print("\nFirst DataFrame (df1) Info:")
df1.info()

print("\nSecond DataFrame (df2) Head:")
print(df2.head())
print("\nSecond DataFrame (df2) Info:")
df2.info()

First DataFrame (df1) Head:
   BIB  COD  ERG    FAN GJAH    LUK  MYR        NUS  PKD  RAS  id
0  160  iii  www   80.0  iii    5.0  eee   800000.0  xxx    t   0
1  153  uuu  aaa  200.0  rrr    0.0  mmm  2000000.0  xxx  NaN   1
2    5  iii  www   96.0  iii   19.0   hh   960000.0   hh    t   2
3    9  iii  www    0.0  iii  120.0  kkk        0.0  qqq  NaN   3
4   40  iii  www  232.0  iii    0.0  mmm  2320000.0  xxx    f   4

First DataFrame (df1) Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4070 entries, 0 to 4069
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   BIB     4070 non-null   int64  
 1   COD     4070 non-null   object 
 2   ERG     4006 non-null   object 
 3   FAN     3966 non-null   float64
 4   GJAH    4006 non-null   object 
 5   LUK     4070 non-null   float64
 6   MYR     4004 non-null   object 
 7   NUS     3966 non-null   float64
 8   PKD     4004 non-null   object 
 9   RAS     1705 non-null   obje

After having a quick look at the information printed above, we can see that both files contain different features that don't overlap besides `id`.
We also see that this is really not a lot of data, something like 4000 samples, and some features are missing data.

After digging into the data, I can also confirm that the types are correctly parsed and that the data is not corrupted at first glance, so there is no apparent need to clean the data in that sense.

In [3]:
# Check if both dataframes contain the same ids before exploring strategies to combine them
a = df1.id.unique()
b = df2.id.unique()

np.array_equal(a, b) 


True

In [4]:
# Check if there are duplicates and remove them
df1.drop_duplicates(inplace=True)
df2.drop_duplicates(inplace=True)

df1.info()
df2.info()


<class 'pandas.core.frame.DataFrame'>
Index: 3700 entries, 0 to 3699
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   BIB     3700 non-null   int64  
 1   COD     3700 non-null   object 
 2   ERG     3636 non-null   object 
 3   FAN     3600 non-null   float64
 4   GJAH    3636 non-null   object 
 5   LUK     3700 non-null   float64
 6   MYR     3634 non-null   object 
 7   NUS     3600 non-null   float64
 8   PKD     3634 non-null   object 
 9   RAS     1555 non-null   object 
 10  id      3700 non-null   int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 346.9+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 3700 entries, 0 to 3699
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SIS     3700 non-null   float64
 1   TOK     3700 non-null   object 
 2   UIN     3661 non-null   float64
 3   VOL     3700 non-null   object 
 4   WET     3700 non-null   i

Apparently, there are 3700 different ids both present in both csv files.

It also seems that there were duplicates in both, but they were exactly the same entries because after removing the duplicates, the data has the same number of entries, 3700 each. This is not common, we were lucky this time ;) 

## Combine Data

In [5]:
# Merge the two dataframes using the 'id' column
# We'll use an inner merge by default, keeping only rows where 'id' exists in both dataframes.
merged_df = pd.merge(df1, df2, on='id')

# Display the first few rows and info of the merged dataframe
print("\nMerged DataFrame Head:")
print(merged_df.head())
print("\nMerged DataFrame Info:")
merged_df.info()

# Check for the number of unique IDs to ensure the merge was as expected
print(f"\nNumber of rows in df1: {len(df1)}")
print(f"Number of rows in df2: {len(df2)}")
print(f"Number of rows in merged_df: {len(merged_df)}")
print(f"Number of unique IDs in merged_df: {merged_df['id'].nunique()}")


Merged DataFrame Head:
   BIB  COD  ERG    FAN GJAH    LUK  MYR        NUS  PKD  RAS  id    SIS TOK  \
0  160  iii  www   80.0  iii    5.0  eee   800000.0  xxx    t   0  1.750   t   
1  153  uuu  aaa  200.0  rrr    0.0  mmm  2000000.0  xxx  NaN   1  0.290   f   
2    5  iii  www   96.0  iii   19.0   hh   960000.0   hh    t   2  0.000   f   
3    9  iii  www    0.0  iii  120.0  kkk        0.0  qqq  NaN   3  0.335   f   
4   40  iii  www  232.0  iii    0.0  mmm  2320000.0  xxx    f   4  0.500   t   

     UIN VOL  WET  KAT XIN Class  
0  17.92   f    1  ccc   t     n  
1  16.92   f    0  ddd   f     n  
2  31.25   f    1  ddd   t     n  
3  48.17   f    0  ccc   f     n  
4  32.33   f    0  ddd   f     n  

Merged DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3700 entries, 0 to 3699
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   BIB     3700 non-null   int64  
 1   COD     3700 non-null   object 
 2   E

The merged dataset also contains 3700 samles as expected, so the merging went smooth.

As seen before some features are missing data, the Class column (target) is not missing data.

# Data Preprocessing

Before building the classification model, we need to preprocess the data. The steps typically include:

1.  **Handling Missing Values:** Identify and decide how to handle any missing data (e.g., imputation, removal).
2.  **Feature Selection/Engineering:** Decide which features to use. The `id` column is likely not useful for prediction and should be dropped. We might also create new features if needed.
3.  **Encoding Categorical Features:** Convert any non-numeric features into a numeric format suitable for machine learning models (e.g., one-hot encoding).
4.  **Feature Scaling:** Scale numerical features to have a similar range (e.g., using StandardScaler) to prevent features with larger values from dominating the model.
5.  **Splitting Data:** Divide the dataset into training and testing sets to evaluate the model's performance on unseen data.

In [6]:
processed_df = merged_df.copy()

# Separate features (X) and target (y)
y = processed_df['Class']
X = processed_df.drop(['Class', 'id'], axis=1)



In [7]:
# Since we have some missing values, we can try to impute them to rescue some.

# This step might be revisited in the future based on the model taken, some algorithms like XGBoost 
# support missing data and making imputations might be detrimental for the model in production in 
# cases there the missing data is actually not random.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nTraining set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")


Training set shape: (2960, 17)
Test set shape: (740, 17)


In [8]:
numeric_features = X_train.select_dtypes(exclude=['object']).columns
categorical_features = X_train.select_dtypes(exclude=['number']).columns

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features),
])

transformed_data_train = preprocessor.fit_transform(X_train)
transformed_data_test = preprocessor.transform(X_test)

# Get feature names
cat_feature_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)
feature_names = list(numeric_features) + list(cat_feature_names)

# Create transformed DataFrame
train_df = pd.DataFrame(transformed_data_train, columns=feature_names)
test_df = pd.DataFrame(transformed_data_test, columns=feature_names)

print(f"\nTraining set shape: {train_df.shape}")
print(f"Test set shape: {test_df.shape}")

train_df.head()


Training set shape: (2960, 53)
Test set shape: (740, 53)


,BIB,FAN,LUK,NUS,SIS,UIN,WET,COD_iii,COD_rrr,COD_uuu,...,RAS_t,TOK_f,TOK_t,VOL_f,VOL_t,KAT_ccc,KAT_ddd,KAT_missing,XIN_f,XIN_t
0,-0.637663,-0.027311,-0.256038,-0.027311,-0.712991,-0.962957,-0.463612,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,0.210390,-0.544392,-0.103849,-0.544392,-0.790900,-0.783750,-0.612822,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
2,1.058442,-1.061473,-0.256038,-1.061473,0.924270,0.118656,-0.612822,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.033712,-0.544392,-0.250397,-0.544392,-0.722294,-0.956585,-0.463612,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,-0.248972,-0.440976,-0.256038,-0.440976,-0.625779,-1.082428,0.431652,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


In [9]:
# Combine features and target for train/test sets
train_df['Class'] = y_train.values 
test_df['Class'] = y_test.values

# Add split indicator
train_df['split'] = 'train'
test_df['split'] = 'test'

# Concatenate train and test sets
final_processed_df = pd.concat([train_df, test_df], ignore_index=False) 

# Define output path
output_parquet_path = 'data/preprocessed_data.parquet'

# Save to Parquet
print(f"\nSaving preprocessed data to {output_parquet_path}...")
try:
    # Ensure the target 'Class' column is treated appropriately if it's not numeric (e.g., convert 'y'/'n' to 1/0 if needed before saving)
    # Example: final_processed_df['Class'] = final_processed_df['Class'].map({'y': 1, 'n': 0})
    # Check dtypes before saving
    print("\nFinal DataFrame Info before saving:")
    final_processed_df.info()
    
    final_processed_df.to_parquet(output_parquet_path, index=True) # Save with index
    print(f"Successfully saved preprocessed data to {output_parquet_path}.")
except ImportError:
    print("\nError: 'pyarrow' or 'fastparquet' package is required to write to Parquet format.")
    print("Please install it using: pip install pyarrow")
except Exception as e:
    print(f"\nAn error occurred while saving to Parquet: {e}")


Saving preprocessed data to data/preprocessed_data.parquet...

Final DataFrame Info before saving:
<class 'pandas.core.frame.DataFrame'>
Index: 3700 entries, 0 to 739
Data columns (total 55 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BIB           3700 non-null   float64
 1   FAN           3700 non-null   float64
 2   LUK           3700 non-null   float64
 3   NUS           3700 non-null   float64
 4   SIS           3700 non-null   float64
 5   UIN           3700 non-null   float64
 6   WET           3700 non-null   float64
 7   COD_iii       3700 non-null   float64
 8   COD_rrr       3700 non-null   float64
 9   COD_uuu       3700 non-null   float64
 10  ERG_aaa       3700 non-null   float64
 11  ERG_missing   3700 non-null   float64
 12  ERG_nnn       3700 non-null   float64
 13  ERG_www       3700 non-null   float64
 14  GJAH_ii       3700 non-null   float64
 15  GJAH_iii      3700 non-null   float64
 16  GJAH_missing  3700 n